## Setup enviroment

In [1]:
import os
import gc
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss, roc_auc_score, average_precision_score, brier_score_loss
import joblib
from venn_abers import VennAbersCalibrator
import sys
sys.path.append(r'C:\Users\jeanr\Documents\projetos\classificacao\lending-club\global')
from util import *
import json

c:\Users\jeanr\Documents\projetos\classificacao\lending-club\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read Test ABT

In [2]:
test_df = pd.read_parquet('../data/processed/abt_test.parquet')

In [3]:
selected_features = None
with open('../artifacts/selected_features_list.json', 'r') as f:
    selected_features = json.load(f)
print(selected_features)

['loan_amnt', 'term', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_record', 'revol_bal', 'total_acc', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_il_tl', 'num_op_rev_tl', 'num_rev_tl_bal_gt_0', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bc_limit', 'income_to_loan_ratio', 'debt_to_income_ratio', 'loan_amnt_per_income', 'fico_avg', 'credit_utilization_ratio', 'delinquency_ratio', 'int_rate_to_income_ratio', 'public_records_impact', 'pct_active_bc', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'application_type']


In [4]:
X_test = test_df[selected_features]

In [5]:
X_test.shape

(538826, 51)

In [6]:
categorical_column_names = X_test.select_dtypes(include=['category']).columns
categorical_column_names

Index(['grade', 'sub_grade', 'emp_length', 'home_ownership',
       'verification_status', 'application_type'],
      dtype='object')

In [7]:
cat_features = ['grade', 'sub_grade', 'emp_length', 'home_ownership',
       'verification_status', 'application_type']

## Loading and aplly the calibrated_model_sigmoid

In [10]:
# Carregar o modelo salvo
calibrated_model_sigmoid = joblib.load("../artifacts/calibrated_model_sigmoid.pkl")

In [11]:
y_test_pred = calibrated_model_sigmoid.predict_proba(X_test)[:, 1]

In [13]:
y_test_pred

array([0.22098969, 0.30973287, 0.35494464, ..., 0.26486728, 0.15503898,
       0.39255849])

In [19]:
submission = test_df.reset_index()[["index"]].copy()  # Pegando o índice como coluna
submission.rename(columns={"index": "Id"}, inplace=True)  # Renomeando para "Id"

# Adicionando as previsões
submission["default"] = y_test_pred

In [20]:
submission

,Id,default
0,472065,0.220990
1,472066,0.309733
2,472067,0.354945
3,472068,0.206459
4,472069,0.429227
...,...,...
538821,1082977,0.097605
538822,1082978,0.043044
538823,1082979,0.264867
538824,1082980,0.155039


In [21]:
submission.to_parquet('../data/processed/jean_submission.parquet', index=False)